# Generator V1

Ferramenta que pega no conteudo das noticias (em formato json) e gera posts.
Baseia-se nas seguintes regras:

        1) A avaliaçao do deve ficar no inicio do post em letras garrafais.
        2) Em seguida deve ser colocada a alegaçao.
        3) Deve depois aparecer hashtags e emojis associados;


In [87]:
import os
import re
import json
from random import choice
import pandas as pd
import random
        
from keybert import KeyBERT
import emoji #https://carpedm20.github.io/emoji/

### Emojis

In [2]:
# A classificaçao por sentimentos foi feita no paper: 
#https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0144296
import pandas as pd
import requests

url = "http://kt.ijs.si/data/Emoji_sentiment_ranking/index.html"

r = requests.get(url)
df_list = pd.read_html(r.text) # this parses all the tables in webpages to a list
df = df_list[0]
df.columns = [c.replace(' ', '_') for c in df.columns]
df = df[df['Occurrences[5...max]']> 400]


df

,Char,Image[twemoji],Unicodecodepoint,Occurrences[5...max],Position[0...1],Neg[0...1],Neut[0...1],Pos[0...1],Sentiment_score[-1...+1],Sentiment_bar(c.i._95%),Unicode_name,Unicode_block
0,😂,😂,0x1f602,14622,0.805,0.247,0.285,0.468,0.221,NaN,FACE WITH TEARS OF JOY,Emoticons
1,❤,❤,0x2764,8050,0.747,0.044,0.166,0.790,0.746,NaN,HEAVY BLACK HEART,Dingbats
2,♥,♥,0x2665,7144,0.754,0.035,0.272,0.693,0.657,NaN,BLACK HEART SUIT,Miscellaneous Symbols
3,😍,😍,0x1f60d,6359,0.765,0.052,0.219,0.729,0.678,NaN,SMILING FACE WITH HEART-SHAPED EYES,Emoticons
4,😭,😭,0x1f62d,5526,0.803,0.436,0.220,0.343,-0.093,NaN,LOUDLY CRYING FACE,Emoticons
...,...,...,...,...,...,...,...,...,...,...,...,...
67,😀,😀,0x1f600,439,0.786,0.086,0.260,0.654,0.568,NaN,GRINNING FACE,Emoticons
68,😚,😚,0x1f61a,424,0.853,0.049,0.192,0.759,0.710,NaN,KISSING FACE WITH CLOSED EYES,Emoticons
69,😻,😻,0x1f63b,417,0.730,0.069,0.243,0.688,0.619,NaN,SMILING CAT FACE WITH HEART-SHAPED EYES,Emoticons
70,©,©,0xa9,416,0.740,0.131,0.621,0.248,0.117,NaN,COPYRIGHT SIGN,Latin-1 Supplement


In [3]:
negative_emojis  = list(df[df['Sentiment_score[-1...+1]'] < 0 ].Char)
positive_emojis = list(df[df['Sentiment_score[-1...+1]'] > 0].Char)

In [4]:
positive_emojis

['😂',
 '❤',
 '♥',
 '😍',
 '😘',
 '😊',
 '👌',
 '💕',
 '👏',
 '😁',
 '☺',
 '♡',
 '👍',
 '🙏',
 '✌',
 '😏',
 '😉',
 '🙌',
 '🙈',
 '💪',
 '😄',
 '💃',
 '💖',
 '😃',
 '😱',
 '🎉',
 '😜',
 '☯',
 '🌸',
 '💜',
 '💙',
 '✨',
 '😳',
 '💗',
 '★',
 '☀',
 '😎',
 '😢',
 '💋',
 '😋',
 '🙊',
 '🎶',
 '💞',
 '😌',
 '🔥',
 '💯',
 '💛',
 '💁',
 '💚',
 '♫',
 '😆',
 '😝',
 '�',
 '😅',
 '👊',
 '😀',
 '😚',
 '😻',
 '©',
 '👀']

In [5]:
emoji.EMOJI_UNICODE

{'en': {':1st_place_medal:': '🥇',
  ':2nd_place_medal:': '🥈',
  ':3rd_place_medal:': '🥉',
  ':AB_button_(blood_type):': '🆎',
  ':ATM_sign:': '🏧',
  ':A_button_(blood_type):': '🅰️',
  ':Afghanistan:': '🇦🇫',
  ':Albania:': '🇦🇱',
  ':Algeria:': '🇩🇿',
  ':American_Samoa:': '🇦🇸',
  ':Andorra:': '🇦🇩',
  ':Angola:': '🇦🇴',
  ':Anguilla:': '🇦🇮',
  ':Antarctica:': '🇦🇶',
  ':Antigua_&_Barbuda:': '🇦🇬',
  ':Aquarius:': '♒',
  ':Argentina:': '🇦🇷',
  ':Aries:': '♈',
  ':Armenia:': '🇦🇲',
  ':Aruba:': '🇦🇼',
  ':Ascension_Island:': '🇦🇨',
  ':Australia:': '🇦🇺',
  ':Austria:': '🇦🇹',
  ':Azerbaijan:': '🇦🇿',
  ':BACK_arrow:': '🔙',
  ':B_button_(blood_type):': '🅱️',
  ':Bahamas:': '🇧🇸',
  ':Bahrain:': '🇧🇭',
  ':Bangladesh:': '🇧🇩',
  ':Barbados:': '🇧🇧',
  ':Belarus:': '🇧🇾',
  ':Belgium:': '🇧🇪',
  ':Belize:': '🇧🇿',
  ':Benin:': '🇧🇯',
  ':Bermuda:': '🇧🇲',
  ':Bhutan:': '🇧🇹',
  ':Bolivia:': '🇧🇴',
  ':Bosnia_&_Herzegovina:': '🇧🇦',
  ':Botswana:': '🇧🇼',
  ':Bouvet_Island:': '🇧🇻',
  ':Brazil:': '🇧🇷',
  ':British_In

In [6]:
## Extrair avaliçoes communs no snopes
evaluations = []
for file in os.listdir("extractors/Snopes/extractions"):
        with open("extractors/Snopes/extractions/" + file) as f:
            data = json.load(f)
            f.close()
        if data['evaluation'] not in evaluations:
            evaluations.append(data['evaluation'])


In [7]:
evaluations

['false',
 'labeled-satire',
 'miscaptioned',
 'true',
 'mostly-false',
 'misattributed',
 'mixture',
 'scam',
 'mostly-true',
 'originated-as-satire',
 'outdated',
 'correct-attribution',
 'unproven',
 'research-in-progress',
 'recall',
 'legit',
 'legend']

In [72]:
dic_emojis = {'false' : negative_emojis ,
 'true' : positive_emojis ,
 'labeled-satire' : negative_emojis,
 'miscaptioned' : negative_emojis,
 'mostly-false' : negative_emojis,
 'misattributed' : negative_emojis,
 'mixture' : negative_emojis,
 'scam' : negative_emojis,
 'mostly-true': positive_emojis,
 'originated-as-satire' : negative_emojis,
 'outdated' : negative_emojis,
 'correct-attribution' : negative_emojis,
 'unproven' : negative_emojis,
 'research-in-progress' : positive_emojis,
 'recall' : negative_emojis,
 'legit' : negative_emojis,
 'legend' : negative_emojis}

### Gerador

In [73]:
#pip install import-ipynb
import hashtags_generetor

In [74]:
from keyphrase_vectorizers import KeyphraseCountVectorizer


In [75]:
def generate(data):
    
    text = re.sub(r'[^\w\s]', '', data['allegation']) 
    words_allegation = [word.lower() for word in text.strip('\n').split() ]
    
    
    ## Definir o modelo
    kw_model = KeyBERT()
    words_bert = [word for word, value in kw_model.extract_keywords(docs=text, keyphrase_ngram_range=(1,1))]
                                        
    allegation =  []
    
    for word in text.strip('\n').split():
        if word.lower() in words_bert:
            allegation.append(word.upper())
        else:
            allegation.append(word)
    
    allegation = ' '.join(allegation)


    #emoji.emojize(':thumbs_up:')
    post = emoji.emojize(':double_exclamation_mark:') + data['evaluation'].upper() + emoji.emojize(':double_exclamation_mark:') + ' '+ allegation 
    #+ choice(dic[data['evaluation']])
    return post

In [90]:
def main():
    posts = {}
    ##### Read the json files with de news
    for file in os.listdir("extractors/Snopes/extractions")[:5]:
        with open("extractors/Snopes/extractions/" + file) as f:
            data = json.load(f)
            f.close()
            
        x = generate(data)
        


        '''
        Identificar as hashtgs a adicionar.

        1) Identificar keywords do post;
        2) Verificar a similaridade dessas keywords;
        3) 
        '''
       
        hashtags = hashtags_generetor.generateHashtags(' '.join(data['postText']).split('\n'))
        hashtags = [' #'+h for h in hashtags]
        post = x + ' '.join (hashtags) + choice(dic_emojis[data['evaluation']]) 
        
        seed = random.randint(0,5)
        
        while (len(post)<280 and seed > 0):
                seed = seed - 1
                post = post + choice(dic_emojis[data['evaluation']]) 
        print(post)


        df = pd.read_csv('v1/snopes_posts.csv')

        row = {'id': data['id'], 'post': post}

        # Adicionar a nova linha ao dataset
        df = df.append(row,  ignore_index=True)

        #Retirar colunaas que estão a mais
        df = df[['id','post' ]]


        df.to_csv('v1/snopes_posts_v1.1.csv')
            

        f.close()
        
        

        
        

In [91]:
main()

‼️FALSE‼️ TWITTERS BAN on former US President DONALD TRUMP was reversed in April 2022 by Elon MUSK #account😪
‼️LABELED-SATIRE‼️ The TWITTER account for the PARODY NEWS site The ONION has been BANNED from TWITTER #account😞
‼️MISCAPTIONED‼️ A video shows an ELECTRICPOWERED BASIGO BUS bursting into FLAMES on Karen Road in KENYA #electric  #fire  #misleading😫
‼️TRUE‼️ The FBI compiled an internal 83PAGE DOCUMENT on INTERNET SLANG that they released in 2014 #communication💪💪👍©
‼️FALSE‼️ A VIDEO shows what a HOT DOG LOOKS like under a MICROSCOPE #microscope😩😫😒😒🔫😒
